Merge Wikipedia-dog.xml and Wikipedia-fish.xml into csv and db files with columns (title, url) for webpage

In [1]:
import sys, os
from xml.dom import minidom
import pandas as pd

In [2]:
os.chdir('..\data')
xmlfile = 'Wikipedia-dog.xml'

In [3]:
def get_page_type(title):
    """
    from WikiPage.py: extract page type from article title
    """
    if 'Category:' in title:
        return 'category'
    elif 'Portal:' in title:
        return 'portal'
    elif 'List of' in title:
        return 'list'
    elif 'File:' in title:
        return 'file'
    else:
        return 'article'

def xml_to_df(xmlfile):
    """
    input: xml filename
    output: data frame with columns: title, url
    
    filter out pages that are not articles
    """
    xmldoc = minidom.parse(xmlfile)
    titlelist = xmldoc.getElementsByTagName('title')
    
    titles = [title.childNodes[0].data for title in titlelist]
    urllist = ['https://en.wikipedia.org/wiki/%s' % (title.replace(' ', '_'))
              for title in titles]
    typelist = [get_page_type(title) for title in titles]
    
    data = [(title, url) for title, url, page_type in zip(titles, urllist, typelist) if page_type == 'article']
    
    data_df = pd.DataFrame(data, columns=['title', 'url'])
    return(data_df)

In [4]:
df1 = xml_to_df(xmlfile)
df1.tail()

,title,url
89,Cat,https://en.wikipedia.org/wiki/Cat
90,Cat bite,https://en.wikipedia.org/wiki/Cat_bite
91,Ancylostoma tubaeforme,https://en.wikipedia.org/wiki/Ancylostoma_tuba...
92,Feline diseases,https://en.wikipedia.org/wiki/Feline_diseases
93,International Cat Day,https://en.wikipedia.org/wiki/International_Ca...


In [5]:
xmlfile = 'Wikipedia-fish.xml'
df2 = xml_to_df(xmlfile)
df2.tail()

,title,url
14,Whale feces,https://en.wikipedia.org/wiki/Whale_feces
15,Whale watching in New Zealand,https://en.wikipedia.org/wiki/Whale_watching_i...
16,Tail sailing,https://en.wikipedia.org/wiki/Tail_sailing
17,Bubble net feeding,https://en.wikipedia.org/wiki/Bubble_net_feeding
18,Ethelbert (whale),https://en.wikipedia.org/wiki/Ethelbert_(whale)


In [6]:
df = df1.append(df2)

In [7]:
df.to_csv('wikisearch.csv', index=False)

In [8]:
import csv
import sqlite3

conn = sqlite3.connect('wikisearch.db')
cur = conn.cursor()
cur.execute("""DROP TABLE IF EXISTS wikisearch""")
cur.execute("""CREATE TABLE wikisearch
            (title text, url text)""")

with open('wikisearch.csv', 'r') as f:
    reader = csv.reader(f.readlines()[1:])  # exclude header line
    cur.executemany("""INSERT INTO wikisearch VALUES (?,?)""",
                    (row for row in reader))
conn.commit()
conn.close()